In [28]:
#hide
#default_exp tools.metric_visualization
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Metric visualization

> Visualize metrics collected from experiments.

In [29]:
#export
import warnings
warnings.filterwarnings('ignore')

import argparse
import sys
sys.path.append('.')
sys.path.append('src')
import pandas as pd

import hpsearch.visualization.experiment_visualization as ev
from hpsearch.config.hpconfig import (get_experiment_manager, get_path_experiments, get_em_args,
                                      add_em_args)
import hpsearch.config.hp_defaults as dflt
from hpsearch.tools.utils import include_best_and_last_experiment, set_metric

In [30]:
#for tests
import pytest
import os

from dsblocks.utils.utils import remove_previous_results

from hpsearch.examples.dummy_experiment_manager import generate_data

## metric visualization

In [32]:
#export
def metric_visualization (experiments=[-1,-2], run_number=0, parameters=None, 
                          name_file='model_history.pk', visualization_options={},
                          manager_path=dflt.manager_path, backend=dflt.backend,
                          metric=None, **kwargs):
    em = get_experiment_manager (manager_path=manager_path, backend=backend, **kwargs)
    folder = em.folder
    metric = set_metric (em, metric)
    op = em.op
    backend = em.backend
    
    # metrics
    metrics = metric
    if not isinstance (experiments, list): experiments = [experiments]
    if isinstance (run_number, range): run_number = list(run_number)
    if not isinstance (run_number, list): run_number = [run_number]
        
    experiments = include_best_and_last_experiment (metrics, experiments=experiments, 
                                                    run_number=run_number[0], op=op,
                                                    manager_path=manager_path)
    
    visualization_options = visualization_options.copy()
    visualization_options.update(kwargs)
    if 'visualization' in visualization_options.keys():
        visualization = visualization_options.pop('visualization')
    else:
        visualization = 'history'
    
    ev.visualize_experiments(visualization=visualization, 
                             experiments=experiments, run_number=run_number, 
                             metrics=metrics, metric=metrics[0], parameters=parameters, 
                             name_file=name_file, **visualization_options, backend=backend)

### Basic usage

In [33]:
#export tests.tools.test_metric_visualization
def test_metric_visualization ():
    em = generate_data ('metric_visualization')
    
    print ('default visualization')
    metric_visualization()
    
    print ('visualizing test_accuracy, and experiments -2 and 0')
    metric_visualization(experiments=[-2, 0], metric='test_accuracy', manager_path=em.manager_path)
    
    print ('using matplotlib as backend')
    metric_visualization(experiments=[3, 7], metric='test_accuracy', backend='matplotlib', 
                         manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [34]:
tst.run (test_metric_visualization, tag='dummy')

running test_metric_visualization
default visualization


,parameters,validation_accuracy
,epochs,
,,
8,30.0,1.0
5,15.0,1.0


visualizing test_accuracy, and experiments -2 and 0


TypeError: hpsearch.visualization.experiment_visualization.visualize_experiments() got multiple values for keyword argument 'metric'

### Using more than one visualization routine

In [ ]:
#export tests.tools.test_metric_visualization
def test_several_visualizations ():
    em = generate_data ('several_visualizations')
    
    metric_visualization(experiments=[3, 7], metric='test_accuracy', backend='matplotlib', 
                         visualization_options={'visualization': ['history', 'metric_correlation', 
                                                                  'custom'],
                                                'metric_1': 'validation_accuracy',
                                                'metric_2': 'test_accuracy'},
                         manager_path=em.manager_path)
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_several_visualizations, tag='dummy')

### Visualize more than one metric

In [ ]:
#export tests.tools.test_metric_visualization
def test_several_metrics ():
    em = generate_data ('several_metrics')
    
    metric_visualization (experiments=[-1,-2], metric=['test_accuracy', 'validation_accuracy'], 
                          backend='matplotlib', manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_several_metrics, tag='dummy')

### visualize more than one metric in same plot

In [ ]:
#export tests.tools.test_metric_visualization
def test_several_metrics_same_plot ():
    em = generate_data ('several_metrics_same_plot')
    
    metric_visualization (experiments=[-1], metric=['validation_accuracy'], 
                          metrics_second=['test_accuracy'],
                          backend='matplotlib', manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_several_metrics_same_plot, tag='dummy')

## parse_args

In [ ]:
#export
def parse_args(args):
    parser = argparse.ArgumentParser(description='show metrics about experiments') 
    # Datasets
    parser.add_argument('-e', nargs='+', default=[-1, -2], type=int,
                        help="experiments")
    parser.add_argument('-m', '--metric', nargs='+', type=str, default=None, help="metrics")
    parser.add_argument('-l','--labels',nargs='+', default=None, type=str)
    parser.add_argument('--run', default=0, type=int)
    parser.add_argument('-f', '--file', default='model_history.pk', type=str)
    parser.add_argument('-v', '--visualization', default='{}', type=str)
    add_em_args (parser, but='metric')

    pars = parser.parse_args(args)
    
    pars.visualization = eval(pars.visualization)
    
    return pars

def parse_arguments_and_visualize (args):
    pars = parse_args(args)
    metric = pars.metric
    del pars.metric
    metric_visualization (pars.e, metric=metric, run_number=pars.run, parameters=pars.labels, 
                          name_file=pars.file, visualization_options=pars.visualization, 
                          **get_em_args (pars))
    
def main():
    parse_arguments_and_visualize (sys.argv[1:])

### Command line example

In [ ]:
#export tests.tools.test_metric_visualization
def test_parse_arguments_and_visualize ():
    em = generate_data ('parse_arguments_and_visualize')

    visualization_options = "{'visualization': ['history', 'metric_correlation', 'custom'], " \
                         "'metric_1': 'validation_accuracy', 'metric_2': 'test_accuracy'}"

    command = '-e 3 7 -m test_accuracy --backend matplotlib -v '.split ()
    command += [f'{visualization_options}']
    command += f'--manager_path {em.manager_path}'.split()

    parse_arguments_and_visualize (command)
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_parse_arguments_and_visualize, tag='dummy')

### Changing the experiment manager properties 

In [ ]:
#export tests.tools.test_metric_visualization
def test_parse_with_em_args_visualization ():
    em = generate_data ('parse_with_em_args_visualization')
    os.rename ('test_parse_with_em_args_visualization', 'test_other_parse_with_em_args_visualization')

    command = ('-e 3 7 -m test_accuracy --backend matplotlib '
               '--parent_path test_other_parse_with_em_args_visualization').split()
    command += f' --manager_path {em.manager_path}'.split()

    parse_arguments_and_visualize (command)
    
    remove_previous_results ('test_other_parse_with_em_args_visualization')
    remove_previous_results ('test_parse_with_em_args_visualization')

In [ ]:
tst.run (test_parse_with_em_args_visualization, tag='dummy', do=True)